In [1]:
import numpy as np
import pandas as pd
import os
import sys
import numpy as np
import yaml
import cv2
from scipy.interpolate import interp1d
from bisect import bisect

In [2]:
scene = "home"
sequence= 1
sample = scene+"1-"+str(sequence)

inpath = os.path.join("/media/prakhar/BIG_BAG/Capstone/"+scene, sample)
print("Loading dataset from: "+inpath)
assert os.path.exists(inpath), "Path DNE: "+inpath

rgbpath = os.path.join(inpath,"color.txt")
depthpath = os.path.join(inpath,"depth.txt")
gyropath = os.path.join(inpath,"d400_gyroscope.txt")
accpath = os.path.join(inpath,"d400_accelerometer.txt")


assoc_path = "/home/prakhar/CMU/Capstone/code/ORB_SLAM3/Examples/Monocular-Inertial/"

outpath = "/media/prakhar/BIG_BAG/Capstone/Loris_monoi/"+sample
#associations file

sensorpath = "/media/prakhar/BIG_BAG/Capstone/"+scene+"/"+sample+"/sensors.yaml"
matpath = "/media/prakhar/BIG_BAG/Capstone/"+scene+"/"+sample+"/trans_matrix.yaml"

timestamp_dir = "/home/prakhar/CMU/Capstone/code/ORB_SLAM3/Examples/Monocular-Inertial/Loris_TimeStamps/"
timestamp_path = os.path.join(timestamp_dir, sample+".txt")

imu_dir = "/home/prakhar/CMU/Capstone/code/ORB_SLAM3/Examples/Monocular-Inertial/Loris_IMU/"
imu_path = os.path.join(imu_dir, sample+".txt")

if not os.path.exists(timestamp_dir):
    os.mkdir(timestamp_dir)
if not os.path.exists(imu_dir):
    os.mkdir(imu_dir)

Loading dataset from: /media/prakhar/BIG_BAG/Capstone/home/home1-1


In [3]:
sensor_data = cv2.FileStorage(sensorpath, cv2.FILE_STORAGE_READ)
mat_data = cv2.FileStorage(matpath, cv2.FILE_STORAGE_READ)

In [4]:
intr_acc = sensor_data.getNode("d400_accelerometer").getNode("imu_intrinsic").mat().reshape(3,4)
intr_gyro = sensor_data.getNode("d400_gyroscope").getNode("imu_intrinsic").mat().reshape(3,4)
print(intr_acc.shape, intr_gyro.shape)
print(intr_acc)
print(intr_gyro)



(3, 4) (3, 4)
[[ 1.01299989e+00  1.62997451e-02 -1.65678188e-02 -2.38034371e-02]
 [ 9.05588327e-04  1.01791656e+00 -8.24022759e-03 -9.57686007e-02]
 [-2.26753242e-02  6.72622863e-03  1.01643240e+00  2.40073770e-01]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -6.76363852e-05]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -9.54246752e-06]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -1.75042805e-05]]


In [5]:
def parse_block(block):
    k1 = block[0].split(": ")[1]
    k2 = block[1].split(": ")[1]
    k3 = "".join(block[6:13]).replace(" ", "").split("data:")[1].replace("[","").replace("]","").split(",")
    k3 = np.array([float(l) for l in k3]).reshape((4,4))
    return k1,k2,k3

def get_trans_mats(matpath):
    with open(matpath,"r") as fp:
        data = [l[:-1] for l in fp.readlines()]
    # print(data)
    ls_blocks = []
    lsl = None
    for l in data:
        if '   -' == l:
            if lsl is not None:
                ls_blocks.append(lsl)
            lsl = []
        else:
            if lsl is not None:
                lsl.append(l)
    if lsl is not None:
        ls_blocks.append(lsl)

    ls_blocks = [parse_block(block) for block in ls_blocks]
    dict_trans = dict()
    for block in ls_blocks:
        k1,k2,k3 = block
        print(k1)
        dict_trans[k1] = dict()
    for block in ls_blocks:
        k1,k2,k3 = block
        dict_trans[k1][k2] = k3
    return dict_trans

In [6]:
dict_trans = get_trans_mats(matpath)
extr_acc = dict_trans['d400_color_optical_frame']['d400_accelerometer']
extr_gyro = dict_trans['d400_color_optical_frame']['d400_gyroscope']

base_link
base_link
base_link
d400_color_optical_frame
d400_color_optical_frame
d400_color_optical_frame
t265_fisheye1_optical_frame
t265_fisheye1_optical_frame
t265_fisheye1_optical_frame


In [20]:
print(extr_acc)
print(extr_gyro)

assert np.allclose(extr_acc, extr_gyro)
print(list(np.linalg.inv(extr_acc).flatten()))

[[ 0.99997545  0.00384914  0.00585471  0.02031279]
 [-0.00382868  0.99998654 -0.00350194  0.00510325]
 [-0.00586812  0.00347944  0.99997673  0.01120139]
 [ 0.          0.          0.          1.        ]]
[[ 0.99997545  0.00384914  0.00585471  0.02031279]
 [-0.00382868  0.99998654 -0.00350194  0.00510325]
 [-0.00586812  0.00347944  0.99997673  0.01120139]
 [ 0.          0.          0.          1.        ]]
[0.9999754529146422, -0.003828680121788734, -0.005868115257977313, -0.02022702517509943, 0.0038491408362146813, 0.999986538707585, 0.0034794422608202006, -0.005220345052741558, 0.005854714594142987, -0.0035019440527251223, 0.9999767290816685, -0.0113021818406191, 0.0, 0.0, 0.0, 1.0]


In [8]:
def get_rgb_data(rgbpath):
    with open(rgbpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
        data = [[float(l[0]), l[0], l[1]] for l in data]
    return data

def get_aligned_depth_data(depthpath):
    with open(depthpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
        data = [[float(l[0]), l[0], 'aligned_'+l[1]] for l in data]
    return data

def get_gyroscope_data(gyropath, intr_gyro):
    with open(gyropath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
    head = data[0]
    data = data[1:]
    data = np.array([np.array([float(k) for k in l]) for l in data])
    datam = data[:,1:]
    data_scaled = np.transpose(intr_gyro[:3,:3]@datam.T) - intr_gyro[:,3]
    data[:,1:] = data_scaled
    return head, data
    
def get_accelerometer_data(accpath, intr_acc):
    with open(accpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
    head = data[0]
    data = data[1:]
    data = np.array([np.array([float(k) for k in l]) for l in data])
    datam = data[:,1:]
    data_scaled = np.transpose(intr_acc[:3,:3]@datam.T) - intr_acc[:,3]
    data[:,1:] = data_scaled
    return head, data


In [9]:
head, data_gyro = get_gyroscope_data(gyropath, intr_gyro)
#take every alternate row of gyro data
data_gyro = data_gyro[::2]
print(head, data_gyro.shape)
head, data_acc = get_accelerometer_data(accpath, intr_acc)
print(head, data_acc.shape)

['#Time', 'Gx', 'Gy', 'Gz'] (30495, 4)
['#Time', 'Ax', 'Ay', 'Az'] (38998, 4)


In [10]:
ind_max = bisect(data_gyro[:,0], data_acc[-1,0])
ind_min = bisect(data_gyro[:,0], data_acc[0,0])
data_gyro = data_gyro[ind_min:ind_max,:]

In [11]:
100/(data_gyro[100,0] - data_gyro[0,0])

199.70203986990313

In [12]:
#create interpolators
acc_interpolator = interp1d(data_acc[:,0], data_acc[:,1:], axis=0)
#interpolate at precise timesteps
acc_interp = acc_interpolator(data_gyro[:,0])

In [13]:
def write_IMU_readings(head, imu_mat, imu_path):
    with open(imu_path, "w") as fp:
        fp.write(head)
        for idr in range(imu_mat.shape[0]):
            reading = imu_mat[idr]
            ts = str(int(reading[0]))
            lsr = [str(x) for x in reading[1:]]
            lsr = [ts] + lsr
            line = ",".join(lsr) + "\n"
            fp.write(line)
    print("Done.")

#imu hed
head = "#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]\n"
imu_mat = np.concatenate([data_gyro, acc_interp], axis=1)
imu_mat[:,0] *= 1e9

write_IMU_readings(head, imu_mat, imu_path)

imu_dir2 = os.path.join(outpath, "mav0/imu0/")

if not os.path.exists(imu_dir2):
    print("Creating dir: "+imu_dir2)
    os.makedirs(imu_dir2)

imu_path2 = os.path.join(imu_dir2, "data.csv")
write_IMU_readings(head, imu_mat, imu_path2)


Done.
Creating dir: /media/prakhar/BIG_BAG/Capstone/Loris_monoi/home1-1/mav0/imu0/
Done.


In [14]:
print(imu_mat[:3])

[[ 1.56000000e+18 -1.74532922e-03  1.39817187e-02 -1.74532928e-03
   5.34399534e-02 -9.84094471e+00  3.82445668e-01]
 [ 1.56000000e+18  0.00000000e+00  1.57270470e-02  0.00000000e+00
   7.32178667e-02 -9.81158382e+00  3.92699442e-01]
 [ 1.56000000e+18  0.00000000e+00  1.39817187e-02  0.00000000e+00
   7.34594319e-02 -9.82572573e+00  3.85158390e-01]]


In [15]:
data_rgb = get_rgb_data(rgbpath)

In [16]:
cam_path = os.path.join(outpath, "mav0/cam0/data/")

if not os.path.exists(cam_path):
    print("Creating dir: "+cam_path)
    os.makedirs(cam_path)

Creating dir: /media/prakhar/BIG_BAG/Capstone/Loris_monoi/home1-1/mav0/cam0/data/


In [17]:
with open(timestamp_path, "w") as fp:
    for idx,line in enumerate(data_rgb):
        tsstr = str(int(line[0]*1e9))
        fp.write(tsstr+"\n")
        fname = tsstr+".png"
        dst = os.path.join(cam_path, fname)
        src = os.path.join(inpath, line[2])
        command = "ln -sf "+src+" "+dst
        os.system(command)
        if (idx+1)%100 == 0:
            print(idx+1,len(data_rgb))

100 4587
200 4587
300 4587
400 4587
500 4587
600 4587
700 4587
800 4587
900 4587
1000 4587
1100 4587
1200 4587
1300 4587
1400 4587
1500 4587
1600 4587
1700 4587
1800 4587
1900 4587
2000 4587
2100 4587
2200 4587
2300 4587
2400 4587
2500 4587
2600 4587
2700 4587
2800 4587
2900 4587
3000 4587
3100 4587
3200 4587
3300 4587
3400 4587
3500 4587
3600 4587
3700 4587
3800 4587
3900 4587
4000 4587
4100 4587
4200 4587
4300 4587
4400 4587
4500 4587


In [ ]:
"/media/prakhar/BIG_BAG/Capstone/Loris_monoi/office1-1/mav0/cam0/data/1560000083949196032.pn"